In [44]:
from scipy.special import comb
import pandas as pd
import numpy as np
import scipy.misc
scipy.misc.comb = comb
import h3
import plotly.express as px


import sys
sys.path.append('project-files')
from helper_functions.gridding import h3_grid

from helper_functions.correlation import correaltion

In [45]:
# Read and concatnate data into a single dataframe
df = pd.read_csv('data/correlation/correlation_new.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148178 entries, 0 to 148177
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ScientificName  148120 non-null  object 
 1   TaxonID         148120 non-null  object 
 2   Date            148178 non-null  object 
 3   Latitude        148024 non-null  float64
 4   Longitude       148024 non-null  float64
 5   Municipality    148052 non-null  object 
dtypes: float64(2), object(4)
memory usage: 6.8+ MB


In [46]:
#Doing the gridding

#Drop NAs from Lat and Lon. Otherwise h3 cannot work with these.
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
df = df[['ScientificName', 'Latitude', 'Longitude']]

#String operations to remove subspecie branching
df['ScientificName'] = df['ScientificName'].str.split(' subsp.').str.get(0)
df['ScientificName'] = df['ScientificName'].str.split(' var.').str.get(0)
  
#Creating separete the dataframes for every specie
df_amanita_muscaria = df.loc[df['ScientificName'] == 'Amanita muscaria']
df_pinus_sylvestris = df.loc[df['ScientificName'] == 'Pinus sylvestris']
df_lepus_timidus = df.loc[df['ScientificName'] == 'Lepus timidus']
df_vulpes_vulpes = df.loc[df['ScientificName'] == 'Vulpes vulpes']
df_taphrina_betulina = df.loc[df['ScientificName'] == 'Taphrina betulina']
df_betula_pubescens = df.loc[df['ScientificName'] == 'Betula pubescens']
df_cantharellus_cibarius = df.loc[df['ScientificName'] == 'Cantharellus cibarius']
df_aegolius_funereus = df.loc[df['ScientificName'] == 'Aegolius funereus']
df_myodes_glareolus = df.loc[df['ScientificName'] == 'Myodes glareolus']
df_microtus_agrestis = df.loc[df['ScientificName'] == 'Microtus agrestis']

#Putting them in a list so we can loop over
specie_list = [df_amanita_muscaria, df_pinus_sylvestris, df_lepus_timidus, df_vulpes_vulpes, df_taphrina_betulina, df_betula_pubescens,
               df_cantharellus_cibarius, df_aegolius_funereus, df_myodes_glareolus, df_microtus_agrestis]

#Fitting all dataframes to grid
for i in range(len(specie_list)):
    specie_df = specie_list[i]
    grid_object = h3_grid()
    grid_object.fit(specie_df)
    grid = grid_object.grid_info()
    grid = grid.drop(columns=['observations_id', 'neighbors'])
    grid = grid.rename(columns={'count':specie_df['ScientificName'].values[0]})
    specie_list[i] = grid


specie_list

/Users/behramulukir/Documents/Data Science Project/DS-Project-2023-Group-2-Reaktor/project-files/helper_functions/gridding.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/behramulukir/Documents/Data Science Project/DS-Project-2023-Group-2-Reaktor/project-files/helper_functions/gridding.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/behramulukir/Documents/Data Science Project/DS-Project-2023-Group-2-Reaktor/project-files/helper_functions/gridding.py:21: SettingWithCopyWarning:


A va

[             h3_cell  Amanita muscaria
 0    8501268bfffffff                 1
 1    850126c7fffffff                 1
 2    850126d3fffffff                 1
 3    850126dbfffffff                 1
 4    85088383fffffff                 1
 ..               ...               ...
 604  85112e7bfffffff                 1
 605  85112e83fffffff                 6
 606  85112e8bfffffff                 5
 607  85112eaffffffff                 1
 608  85112ecbfffffff                 1
 
 [609 rows x 2 columns],
               h3_cell  Pinus sylvestris
 0     85012613fffffff                 2
 1     8501261bfffffff                15
 2     85012643fffffff                 1
 3     85012647fffffff                 2
 4     8501264ffffffff                 1
 ...               ...               ...
 1867  85112ecbfffffff                49
 1868  85112ecffffffff                21
 1869  85112ed3fffffff                22
 1870  85112ed7fffffff                16
 1871  85112edbfffffff                22
 

In [47]:
#Create correlation class to call from helper function
correlation_func = correaltion()

#Merge different dataframes into one for simplicty

combined_df = specie_list[0]
#specie_list = specie_list[1:]

for i in range(len(specie_list)):
    specie_df = specie_list[i]
    combined_df = combined_df.merge(specie_df, how='outer')

combined_df = combined_df.fillna(0)
combined_df

,h3_cell,Amanita muscaria,Pinus sylvestris,Lepus timidus,Vulpes vulpes,Taphrina betulina,Betula pubescens,Cantharellus cibarius,Aegolius funereus,Myodes glareolus,Microtus agrestis
0,8501268bfffffff,1.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
1,850126c7fffffff,1.0,38.0,4.0,0.0,0.0,40.0,0.0,3.0,0.0,0.0
2,850126d3fffffff,1.0,30.0,1.0,1.0,0.0,47.0,0.0,1.0,0.0,0.0
3,850126dbfffffff,1.0,5.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0
4,85088383fffffff,1.0,11.0,0.0,1.0,0.0,10.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1916,850883a3fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1917,850883cbfffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1918,85088883fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1919,8508c873fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0


In [48]:
def one_way_relation(var1, var2, dataframe):
    dataframe_1 = dataframe.loc[dataframe[var1] >= 1]
    var1ToVar2 = dataframe_1.loc[dataframe_1[var2] >= 1].shape[0]
    print('Var1 Count')
    print(dataframe_1.shape[0])
    dataframe_2 = dataframe.loc[dataframe[var2] >= 1]
    print('Var2 Count')
    print(dataframe_2.shape[0])
    var2ToVar1 = dataframe_2.loc[dataframe_2[var1] >= 1].shape[0]
    return var1ToVar2, var2ToVar1

In [49]:
def count_comparsion(varA, varB, dataframe):
    varA_df_1 = dataframe.loc[dataframe[varA] >= 1]
    varA_df_meanA = varA_df_1[varA].mean()
    df_varA1_varB0 = varA_df_1.loc[varA_df_1[varB] == 0]
    varA1_varB0_mean = df_varA1_varB0[varA].mean()
    df_varA1_varB1 = varA_df_1.loc[varA_df_1[varB] >= 1]
    varA1_varB1_mean = df_varA1_varB1[varA].mean()

    varB_df_1 = dataframe.loc[dataframe[varB] >= 1]
    varB_df_meanB = varB_df_1[varB].mean()
    df_varA0_varB1 = varB_df_1.loc[varB_df_1[varA] == 0]
    varA0_varB1_mean = df_varA0_varB1[varB].mean()
    df_varB1_varA1 = varB_df_1.loc[varB_df_1[varA] >= 1]
    varB1_varA1_mean = df_varB1_varA1[varB].mean()
    return varA_df_meanA, varA1_varB0_mean, varA1_varB1_mean, varB_df_meanB, varA0_varB1_mean, varB1_varA1_mean

In [50]:
#Symbiotic case 1: Amanita muscaria vs Pinus sylvestris
symbiotic_df_1 = combined_df[['h3_cell', 'Amanita muscaria', 'Pinus sylvestris']]

#symbiotic_corr = symbiotic_df.corr(method='pearson')
#print(symbiotic_corr)

#print(one_way_relation('Amanita muscaria', 'Pinus sylvestris', symbiotic_df_1))
#print(count_comparsion('Amanita muscaria', 'Pinus sylvestris', symbiotic_df_1))

symbiotic_analysis_1 = correlation_func.relative_location_info('Amanita muscaria', 'Pinus sylvestris', symbiotic_df_1)
for k, v in symbiotic_analysis_1.items():
    print(k,':', v,'\n')

symbiotic_analysis_1_graph = {'Specie 1': symbiotic_analysis_1['species1_overall_mean'], 'Specie 2' : symbiotic_analysis_1['species2_overall_mean'], 'Specie 1 where Specie 2 also exists' :symbiotic_analysis_1['species1_where2_mean'], 'Specie 2 where Specie 1 also exists' :symbiotic_analysis_1['species2_where1_mean']}

symbiotic_analysis_1_bar = px.bar(pd.DataFrame(symbiotic_analysis_1_graph, index=['key']).T, y='key')
symbiotic_analysis_1_bar.update_layout(title="Symbiotic Case: Aegolius funereus vs. Clethrionomys glareolus & Microtus agrestis", xaxis_title=None, yaxis_title='Average number of species')

symbiotic_analysis_1_bar.show()

intersection_count : 606 

species1_count : 609 

species2_count : 1872 

species1_overall_mean : 4.577996715927751 

species2_overall_mean : 43.037393162393165 

species1_where2_mean : 4.594059405940594 

species2_where1_mean : 59.7029702970297 

species1_not2_mean : 43.037393162393165 



In [51]:
#Symbiotic case 2: Cantharellus cibarius vs Pinus sylvestris
symbiotic_df_2 = combined_df[['h3_cell', 'Cantharellus cibarius', 'Pinus sylvestris']]

#symbiotic_corr = symbiotic_df.corr(method='pearson')
#print(symbiotic_corr)

#print(one_way_relation('Cantharellus cibarius', 'Pinus sylvestris', symbiotic_df_2))
#print(count_comparsion('Cantharellus cibarius', 'Pinus sylvestris', symbiotic_df_2))

symbiotic_analysis_2 = correlation_func.relative_location_info('Cantharellus cibarius', 'Pinus sylvestris', symbiotic_df_2)
for k, v in symbiotic_analysis_2.items():
    print(k,':', v,'\n')

intersection_count : 505 

species1_count : 506 

species2_count : 1872 

species1_overall_mean : 3.347826086956522 

species2_overall_mean : 43.037393162393165 

species1_where2_mean : 3.3524752475247523 

species2_where1_mean : 61.43564356435643 

species1_not2_mean : 43.037393162393165 



In [52]:
#Predatory case 1: Lepus timidus vs Vulpes vulpes
predatory_df_1 = combined_df[['h3_cell', 'Lepus timidus', 'Vulpes vulpes']]

#predatory_corr = predatory_df.corr(method='pearson')
#print(predatory_corr)

#print(one_way_relation('Lepus timidus', 'Vulpes vulpes', predatory_df_1))
#print(count_comparsion('Lepus timidus', 'Vulpes vulpes', predatory_df_1))

predatory_analysis_1 = correlation_func.relative_location_info('Lepus timidus', 'Vulpes vulpes', predatory_df_1)
for k, v in predatory_analysis_1.items():
    print(k,':', v,'\n')

intersection_count : 436 

species1_count : 810 

species2_count : 618 

species1_overall_mean : 6.646913580246913 

species2_overall_mean : 7.106796116504855 

species1_where2_mean : 9.243119266055047 

species2_where1_mean : 9.075688073394495 

species1_not2_mean : 7.106796116504855 



In [53]:
#Predatory case 2: Aegolius funereus vs Clethrionomys glareolus (Myodes glareolus) & Microtus agrestis
predatory_df_2 = combined_df[['h3_cell', 'Aegolius funereus', 'Myodes glareolus', 'Microtus agrestis']]

predatory_df_2['Mole'] = predatory_df_2['Myodes glareolus'] + predatory_df_2['Microtus agrestis']
predatory_df_2 = predatory_df_2.drop(columns=['Myodes glareolus', 'Microtus agrestis'])

#predatory_corr = predatory_df.corr(method='pearson')
#print(predatory_corr)

#print(one_way_relation('Aegolius funereus', 'Mole', predatory_df_2))
#print(count_comparsion('Aegolius funereus', 'Mole', predatory_df_2))

predatory_analysis_2 = correlation_func.relative_location_info('Mole', 'Aegolius funereus', predatory_df_2)
for k, v in predatory_analysis_2.items():
    print(k,':', v,'\n')


predatory_analysis_2_graph = {'Specie 1': predatory_analysis_2['species1_overall_mean'], 'Specie 2' : predatory_analysis_2['species2_overall_mean'], 'Specie 1 where Specie 2 also exists' :predatory_analysis_2['species1_where2_mean'], 'Specie 2 where Specie 1 also exists' :predatory_analysis_2['species2_where1_mean']}

predatory_analysis_2_bar = px.bar(pd.DataFrame(predatory_analysis_2_graph, index=['key']).T, y='key')
predatory_analysis_2_bar.update_layout(title="Predatory Case: Aegolius funereus vs. Clethrionomys glareolus & Microtus agrestis", xaxis_title=None, yaxis_title='Average number of species')

predatory_analysis_2_bar.show()

intersection_count : 235 

species1_count : 256 

species2_count : 1267 

species1_overall_mean : 5.7265625 

species2_overall_mean : 20.23204419889503 

species1_where2_mean : 5.770212765957447 

species2_where1_mean : 55.12340425531915 

species1_not2_mean : 20.23204419889503 



/var/folders/hk/_g475vss66n_f_6s0_3sdxk40000gn/T/ipykernel_6549/3319781090.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [54]:
#Parasitic case
parasitic_df = combined_df[['h3_cell', 'Taphrina betulina', 'Betula pubescens']]

#parasitic_corr = parasitic_df.corr(method='pearson')
#print(parasitic_corr)

#print(one_way_relation('Taphrina betulina', 'Betula pubescens', parasitic_df))
#print(count_comparsion('Taphrina betulina', 'Betula pubescens', parasitic_df))


parasitic_analysis = correlation_func.relative_location_info('Taphrina betulina', 'Betula pubescens', parasitic_df)
for k, v in parasitic_analysis.items():
    print(k,':', v,'\n')

parasitic_analysis_graph = {'Specie 1': parasitic_analysis['species1_overall_mean'], 'Specie 2' : parasitic_analysis['species2_overall_mean'], 'Specie 1 where Specie 2 also exists' :parasitic_analysis['species1_where2_mean'], 'Specie 2 where Specie 1 also exists' :parasitic_analysis['species2_where1_mean']}

parasitic_bar = px.bar(pd.DataFrame(parasitic_analysis_graph, index=['key']).T, y='key')
parasitic_bar.update_layout(title="Parasitic Case: Taphrina betulina vs. Betula pubescens", xaxis_title=None, yaxis_title='Average number of species')

parasitic_bar.show()

intersection_count : 71 

species1_count : 71 

species2_count : 1880 

species1_overall_mean : 3.76056338028169 

species2_overall_mean : 13.710106382978724 

species1_where2_mean : 3.76056338028169 

species2_where1_mean : 33.985915492957744 

species1_not2_mean : 13.710106382978724 

